In [ ]:
#install the neccessary package

%pip install pyarrow

In [1]:
#import the libeary
import pandas as pd 
import os 
import requests
from azure.storage.blob import BlobServiceClient
from azure.storage.blob import  BlobClient
import psycopg2
from sqlalchemy import create_engine
import json
import csv
import datetime 
import datetime as dt
import pyarrow
import pyarrow.parquet as pq
from io import BytesIO
import io

In [2]:
#import the api key from rapidminer 


url = "https://realty-mole-property-api.p.rapidapi.com/randomProperties"

querystring = {"limit":"1000"}

headers = {
	"x-rapidapi-key": "89338c3b35mshef78a559ef744efp15f713jsn28765a5b0ba1",
	"x-rapidapi-host": "realty-mole-property-api.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

data =response.json()
#create a dataframe from the data
df = pd.DataFrame(data)

In [3]:
#check the dataframe
df.head(3)

,addressLine1,city,state,zipCode,formattedAddress,squareFootage,yearBuilt,county,assessorID,legalDescription,...,propertyTaxes,owner,id,longitude,latitude,subdivision,bathrooms,lotSize,bedrooms,addressLine2
0,363 Andover Pl,Robbinsville,NJ,08691,"363 Andover Pl, Robbinsville, NJ 08691",1040.0,1987.0,Mercer,12 00005-0000-00025- 02-C363,LOT:25.02 BLK:5 DIST:12 CITY/MUNI/TWP:WASHINGT...,...,{'2021': {'total': 5145}},"{'names': ['Keith S Prince', 'ROSEMARY PRINCE'...","363-Andover-Pl,-Robbinsville,-NJ-08691",-74.634395,40.235946,NaN,NaN,NaN,NaN,NaN
1,400 Jared St,Bentonville,AR,72712,"400 Jared St, Bentonville, AR 72712",1710.0,1994.0,Benton,01-05831-000,PLAT 18-184,...,"{'2019': {'total': 1543}, '2021': {'total': 20...","{'names': ['RONALD G JONES'], 'mailingAddress'...","400-Jared-St,-Bentonville,-AR-72712",-94.178005,36.376712,BLUEBERRY HEIGHTS-BENTONVILLE,2.0,9583.0,3.0,NaN
2,1117 Market St,Colorado Springs,CO,80904,"1117 Market St, Colorado Springs, CO 80904",NaN,NaN,El Paso,NaN,NaN,...,NaN,NaN,"1117-Market-St,-Colorado-Springs,-CO-80904",-104.866935,38.836611,NaN,NaN,NaN,NaN,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 27 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   addressLine1      500 non-null    object 
 1   city              500 non-null    object 
 2   state             500 non-null    object 
 3   zipCode           500 non-null    object 
 4   formattedAddress  500 non-null    object 
 5   squareFootage     396 non-null    float64
 6   yearBuilt         373 non-null    float64
 7   county            500 non-null    object 
 8   assessorID        355 non-null    object 
 9   legalDescription  341 non-null    object 
 10  zoning            203 non-null    object 
 11  propertyType      415 non-null    object 
 12  lastSalePrice     256 non-null    float64
 13  lastSaleDate      308 non-null    object 
 14  ownerOccupied     310 non-null    object 
 15  features          446 non-null    object 
 16  taxAssessment     350 non-null    object 
 1

In [5]:
#fill the missing data 
df=df.fillna({
'addressLine2':'unkown',
'assessorID':'unkown',
'bedrooms':0.0,
'features':'unkown',
'legalDescription':'unkown',
'squareFootage':0.0,
'subdivision':'unkown',
'yearBuilt':0.0,
'bathrooms':0.0,
'lotSize':0.0,
'propertyType':'unkown',
 'taxAssessment':'unkown',
'propertyTaxes':'unkown',
'lastSalePrice':0.0,
'lastSaleDate':'unkown',
'zoning': 'unkown',
'ownerOccupied':'unkown',
'owner':'unkown'       
})

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 27 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   addressLine1      500 non-null    object 
 1   city              500 non-null    object 
 2   state             500 non-null    object 
 3   zipCode           500 non-null    object 
 4   formattedAddress  500 non-null    object 
 5   squareFootage     500 non-null    float64
 6   yearBuilt         500 non-null    float64
 7   county            500 non-null    object 
 8   assessorID        500 non-null    object 
 9   legalDescription  500 non-null    object 
 10  zoning            500 non-null    object 
 11  propertyType      500 non-null    object 
 12  lastSalePrice     500 non-null    float64
 13  lastSaleDate      500 non-null    object 
 14  ownerOccupied     500 non-null    object 
 15  features          500 non-null    object 
 16  taxAssessment     500 non-null    object 
 1

In [7]:
df2=df.copy()

In [8]:
#convert necessary column to string to aviod convertion issue
df['features']=df['features'].apply(lambda x:json.dumps(x))
df['taxAssessment'] = df['taxAssessment'].apply(lambda x:json.dumps(x) if isinstance(x, dict) else str(x))
df['propertyType'] = df['propertyType'].apply(lambda x:json.dumps(x) if isinstance(x, dict) else str(x))
df['owner'] = df['owner'].apply(lambda x:json.dumps(x) if isinstance(x, dict) else str(x))
df['ownerOccupied']=df['ownerOccupied'].apply(lambda x:json.dumps(x) if isinstance(x,dict) else str(x))


In [9]:
df['lastSaleDate']=pd.to_datetime(df['lastSaleDate'], errors='coerce')
df['Date'] =pd.to_datetime(df['lastSaleDate']).dt.date
df['Day'] =pd.to_datetime(df['lastSaleDate']).dt.day_name()
df['Month'] =pd.to_datetime(df['lastSaleDate']).dt.month_name()
df['Year'] =pd.to_datetime(df['lastSaleDate']).dt.year

In [10]:
#create the sales table 
sales =df[['lastSalePrice','Date','Day','Month','Year']].drop_duplicates().reset_index(drop=True)
sales['saleid']=sales.index+1
sales= sales[['saleid','lastSalePrice','Date','Day','Month','Year']].drop_duplicates()
sales

,saleid,lastSalePrice,Date,Day,Month,Year
0,1,107805.0,1988-06-03,Friday,June,1988.0
1,2,87000.0,1997-11-05,Wednesday,November,1997.0
2,3,0.0,NaT,NaN,NaN,NaN
3,4,10000.0,2017-01-25,Wednesday,January,2017.0
4,5,510000.0,2009-07-02,Thursday,July,2009.0
...,...,...,...,...,...,...
302,303,194900.0,2023-01-03,Tuesday,January,2023.0
303,304,579900.0,2016-07-08,Friday,July,2016.0
304,305,0.0,2022-01-18,Tuesday,January,2022.0
305,306,455000.0,2021-01-13,Wednesday,January,2021.0


In [11]:
#create the location table
location = df[['addressLine1', 'city', 'state', 'zipCode', 'formattedAddress','county','longitude','latitude',]].drop_duplicates().reset_index(drop=True)
location.index.name ='locationid'
location=location.reset_index()
location

,locationid,addressLine1,city,state,zipCode,formattedAddress,county,longitude,latitude
0,0,363 Andover Pl,Robbinsville,NJ,08691,"363 Andover Pl, Robbinsville, NJ 08691",Mercer,-74.634395,40.235946
1,1,400 Jared St,Bentonville,AR,72712,"400 Jared St, Bentonville, AR 72712",Benton,-94.178005,36.376712
2,2,1117 Market St,Colorado Springs,CO,80904,"1117 Market St, Colorado Springs, CO 80904",El Paso,-104.866935,38.836611
3,3,1120 San Rafael Ave Apt H,Glendale,CA,91202,"1120 San Rafael Ave Apt H, Glendale, CA 91202",Los Angeles,-118.260452,34.162299
4,4,40 W Glenwood Ave,Akron,OH,44304,"40 W Glenwood Ave, Akron, OH 44304",Summit,-81.517415,41.095131
...,...,...,...,...,...,...,...,...,...
495,495,7635 Broken Hickory Dr,Walls,MS,38680,"7635 Broken Hickory Dr, Walls, MS 38680",DeSoto,-90.102858,34.971820
496,496,9691 Triton Ct,Boca Raton,FL,33434,"9691 Triton Ct, Boca Raton, FL 33434",Palm Beach,-80.198349,26.379862
497,497,2826 Werk Rd,Cincinnati,OH,45211,"2826 Werk Rd, Cincinnati, OH 45211",Hamilton,-84.593579,39.141946
498,498,7171 Vista,La Verne,CA,91750,"7171 Vista, De Oro, La Verne, CA 91750",Los Angeles,-117.754614,34.145009


In [12]:
#create the feature table
feature =df[['features', 'propertyType', 'yearBuilt', 'zoning']].drop_duplicates().reset_index(drop=True)
feature.index.name='featureid'
feature=feature.reset_index()
feature

,featureid,features,propertyType,yearBuilt,zoning
0,0,"{""architectureType"": ""Condo / Apartment"", ""uni...",Condo,1987.0,PRVD
1,1,"{""cooling"": true, ""coolingType"": ""Central"", ""e...",Single Family,1994.0,unkown
2,2,{},unkown,0.0,unkown
3,3,"{""architectureType"": ""Colonial"", ""cooling"": tr...",Single Family,1919.0,unkown
4,4,"{""garage"": true, ""garageSpaces"": 2, ""garageTyp...",Single Family,2005.0,R-1:SINGLE FAM-RES
...,...,...,...,...,...
383,383,"{""architectureType"": ""Condo / Apartment"", ""coo...",Townhouse,1925.0,R-3
384,384,"{""exteriorType"": ""Brick Veneer"", ""floorCount"":...",Single Family,2000.0,R-30
385,385,"{""cooling"": true, ""coolingType"": ""Commercial"",...",Single Family,1985.0,RT
386,386,"{""architectureType"": ""Conventional"", ""cooling""...",Single Family,1928.0,unkown


In [13]:
#create the fact table 
fact_table =df.merge(sales,on=['lastSalePrice','Date','Day','Month','Year'], how ='left')\
    .merge(location, on=['addressLine1', 'city', 'state', 'zipCode', 'formattedAddress','county','longitude','latitude'], how ='left')\
    .merge(feature, on=['features', 'propertyType', 'yearBuilt', 'zoning'],how ='left')

In [14]:
fact_table=fact_table[['saleid', 'locationid', 'featureid','squareFootage', 'yearBuilt', 'county', 'assessorID'\
                    ,'legalDescription', 'subdivision','bathrooms', 'lotSize'\
                        , 'id','bedrooms']].reset_index(drop=True)
fact_table.index.name ='factid'
fact_table=fact_table.reset_index()
fact_table =fact_table[['factid','saleid', 'locationid', 'featureid','squareFootage', 'yearBuilt', 'county', 'assessorID'\
                    ,'legalDescription', 'subdivision','bathrooms', 'lotSize'\
                         , 'id','bedrooms']]

In [15]:
# load data in csv file 
sales.to_csv('sales.csv', index=False)
location.to_csv('location.csv', index=False)
feature.to_csv('feature.csv', index=False)
fact_table.to_csv('fact_table.csv', index=False)
df.to_csv('cleandata.csv', index=False)



In [16]:
import os
from dotenv import load_dotenv
from azure.storage.blob import BlobServiceClient

# Load environment variables from a .env file
load_dotenv()

# Get connection string and container name from environment variables
container_str = os.getenv('AZURE_BLOB_STORAGE_CONNECTION_STRING')
container_name = os.getenv('AZURE_BLOB_STORAGE_NAME')

# Check if the required environment variables are set
if not container_str:
    raise ValueError("AZURE_BLOB_STORAGE_CONNECTION_STRING environment variable not set.")
if not container_name:
    raise ValueError("environment variable not set.")

# Initialize BlobServiceClient
try:
    blob_service_client = BlobServiceClient.from_connection_string(container_str)
except Exception as e:
    raise ConnectionError(f"Failed to create BlobServiceClient: {e}")

# Get a container client
try:
    container_client = blob_service_client.get_container_client(container_name)
except Exception as e:
    raise ConnectionError(f"Failed to get container client: {e}")

# Example usage
print(f"Blob service client and container client created successfully.")


Blob service client and container client created successfully.


In [ ]:
cleandata =pd.read_csv('cleandata.csv')
cleandata

In [19]:
load_dotenv()
connection_string = os.getenv('AZURE_BLOB_STORAGE_CONNECTION_STRING')
container_name = os.getenv('AZURE_BLOB_STORAGE_NAME')

In [ ]:
print(f"Connection String: {connection_string}")
print(f"Container Name: {container_name}")

In [21]:
import os
from azure.storage.blob import BlobServiceClient
import pandas as pd
load_dotenv()

# Load connection string from environment variables
connection_string = os.getenv('AZURE_BLOB_STORAGE_CONNECTION_STRING').strip()

if not connection_string:
    raise ValueError("Azure Storage connection string is missing.")

# Initialize BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# Define the container name
container_name = os.getenv('AZURE_BLOB_STORAGE_NAME').strip()

# Get a container client
container_client = blob_service_client.get_container_client(container_name)

# Check if the container exists, if not create it
if not container_client.exists():
    print(f"Container '{container_name}' does not exist.")
    container_client.create_container()
    print(f"Container '{container_name}' created successfully.")
else:
    print(f"Container '{container_name}' already exists.")

# List of dataframes and corresponding blob names
data = [
    (sales, 'data2/sales_table.csv'),
    (location, 'data2/location_table.csv'),
    (feature, 'data2/feature_table.csv'),
    (fact_table, 'data2/fact_table.csv'),  # Assuming 'date_table' refers to a dataframe
    (cleandata, 'data2/cleandata_table.csv')
]

# Upload each dataframe as a blob to Azure Blob Storage
for df, blob_name in data:
    try:
        # Convert DataFrame to CSV
        csv_data = df.to_csv(index=False)

        # Get a blob client
        blob_client = container_client.get_blob_client(blob_name)

        # Upload CSV data to blob storage
        blob_client.upload_blob(csv_data, overwrite=True)
        print(f"Uploaded {blob_name} to Azure Blob Storage successfully.")

    except Exception as e:
        print(f"An error occurred while uploading {blob_name}: {e}")


Container 'newcontaineropen' already exists.
Uploaded data2/sales_table.csv to Azure Blob Storage successfully.
Uploaded data2/location_table.csv to Azure Blob Storage successfully.
Uploaded data2/feature_table.csv to Azure Blob Storage successfully.
Uploaded data2/fact_table.csv to Azure Blob Storage successfully.
Uploaded data2/cleandata_table.csv to Azure Blob Storage successfully.
